In [1]:
# Importing libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import torch.nn.functional as F


In [2]:

# Loading the CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


100%|██████████| 170498071/170498071 [00:05<00:00, 33663516.26it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
# Defining a basic convolution to use in the model
class BasicConv(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

In [4]:
# Defining the inception model
class InceptionModule(nn.Module):
    def __init__(self, in_channels, n1x1, n3x3red, n3x3, n5x5red, n5x5, pool_proj):
        super(InceptionModule, self).__init__()
        # 1x1 conv branch
        self.branch1 = BasicConv(in_channels, n1x1, kernel_size=1)

        # 3x3 conv branch
        self.branch2 = nn.Sequential(
            BasicConv(in_channels, n3x3red, kernel_size=1),
            BasicConv(n3x3red, n3x3, kernel_size=3, padding=1)
        )

        # 5x5 conv branch
        self.branch3 = nn.Sequential(
            BasicConv(in_channels, n5x5red, kernel_size=1),
            BasicConv(n5x5red, n5x5, kernel_size=5, padding=2)
        )

        # Max pooling branch
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            BasicConv(in_channels, pool_proj, kernel_size=1)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        outputs = [branch1, branch2, branch3, branch4]
        return torch.cat(outputs, 1)


In [5]:
# Defining a modified and simplified version of GoogLeNet
class ModifiedGoogLeNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ModifiedGoogLeNet, self).__init__()

        self.conv1 = BasicConv(3, 64, kernel_size=5, padding=2)
        self.inception1 = InceptionModule(64, 32, 16, 32, 16, 32, 32)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inception2 = InceptionModule(128, 64, 32, 64, 32, 64, 64)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.inception1(x)
        x = self.maxpool(x)
        x = self.inception2(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [6]:
# Setting up training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ModifiedGoogLeNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

def train(model, criterion, optimizer, train_loader, epochs=3):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f}")


In [7]:
# Training the model (run time 4m)
train(model, criterion, optimizer, train_loader, epochs=10)

Epoch 1/10 - Loss: 1.4963
Epoch 2/10 - Loss: 1.1782
Epoch 3/10 - Loss: 1.0562
Epoch 4/10 - Loss: 0.9757
Epoch 5/10 - Loss: 0.9140
Epoch 6/10 - Loss: 0.8682
Epoch 7/10 - Loss: 0.8271
Epoch 8/10 - Loss: 0.7931
Epoch 9/10 - Loss: 0.7633
Epoch 10/10 - Loss: 0.7368


In [8]:
# Evaluating the model on test data
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

test(model, test_loader)

Test Accuracy: 67.59%
